# Presentación del Problema: Identificación binaria de Razas de Perros

Identificar razas de perros es un problema interesante de computer vision debido a las pequeñas diferencias visuales que separan una raza de otra. Durante este ejercicio se trabajará con la base de datos de raza de perros de Kaggle (https://www.kaggle.com/c/dog-breed-identification/data) ya cargados y separados en train y test.

## Tarea del modelo
La tarea a realizar será la clasificación binaria de diferentes imágenes de perros solamente en dos razas distintas.

## Objetivos del ejercicio
El objetivo del ejercicio es que, partiendo de una base de datos, con un preprocesamiento de imágenes y arquitectura de una red dada, los alumnos sean capaces de mejorar el resultado mediante técnicas de optimización, que pueden incluir modificar la arquitectura de la red.

## Estructura del Notebook
La estructura del presente Notebook está dividida en las siguientes secciones

1.   Operaciones iniciales para trabajar con los archivos de Drive
2.   Preprocesamiento de los datos de entrada
3.   Arquitectura de la red propuesta
4.   Propuestas de mejora

## Base de Datos
La base de datos original consta de alrededor de 20.500 imágenes de perros, clasificadas en 120 razas diferentes, divididas al 50 % en training y test.
Durante el ejercicio se trabajará solo con 2 razas de perros elegidas al azar, Golder Retrievers y Pastores escoceses, para enfocarnos en la tarea de clasificación binaria.

Se debe tener en cuenta que las imágenes de la base de datos están etiquetadas solamente por un id, y realiza el mapeo de la raza a través de un archivo CSV.

## Operaciones Iniciales

En primer lugar, al tener las imágenes de entrenamiento y test cargadas en el Drive, este debe ser montado sobre el Notebook para poder ser utilizado, lo que se hace con la celda siguiente.

Al correr la celda aparecerá un link y un campo para ingresar un código de autorización. Este código se obtendrá al darle autorización a Colab para acceder a Drive mediante el Link, por lo que se debe:

1. Hacer click en el Link
2. Copiar el código de autorización
3. Pegarlo en el campo provisto para tal fin
4. Apretar Enter

Luego de estos pasos, los archivos del Drive podrán ser utilizados desde el Notebook.

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


### Navegar hasta el directorio con las carpetas train y test

Con el drive ya montado, es posible acceder a los archivos mediante comandos de consola, por ejemplo:
```
!ls
```
permite listar información sobre los archivos y carpetas del directorio activo


In [0]:
# Permite ver el contenido del directorio activo
!ls

sample_data



```
%cd ruta_objetivo
```
permite navegar por el directorio cambiando de carpeta

Usando estos dos comandos, lo que se debe hacer es:

1. Posicionarse en el directorio donde se encuentran las carpetas *train* y *test*

In [9]:
%cd /gdrive/'My Drive'/'Machine Learning: Fundamentos y Aplicaciones'
!ls

/gdrive/My Drive/Machine Learning: Fundamentos y Aplicaciones
'1-Python, Numpy y MNIST.ipynb'   labeled_train  'Proyecto Final.ipynb'   train
'2- CIFAR10 y CIFAR100.ipynb'	  labels.csv	  test


## Preprocesamiento de los datos de entrada

En primer lugar se debe trabajar con los datos para que sean fácilmente utilizables al momento de realizar el entrenamiento y el test. Para ello se realizará lo siguiente:

1. Crear un diccionario que relacione el id del archivo con la raza correcta.

2. A partir de él, generar un "diccionario contador", para reemplazar cada nombre de archivo de imágenes por su raza y un número que lo acompañe. Por ejemplo, *Golden_Retriever-1*, *Golden_Retriever-2*, y así sucesivamente.

3. Guardar las imágenes con estos nuevos nombres de archivo.

4. Modificar el tamaño de las imágenes para utilizar un tamaño de entrada uniforme.

#### Importar todos los paquetes a utilizar

In [0]:
from PIL import Image           # para cargar las imágenes
import numpy as np
import os                       # para navegar por el sistema de archivos
import imageio                  # para escribir imágenes
from random import shuffle      # para aleatorizar la posición de las imágenes
import matplotlib.pyplot as plt # para mostrar las imágenes

#### Crear el diccionario que relacione el id de la imagen con la raza

In [4]:
naming_dict = {}                                          # id: raza
f = open("labels.csv", "r")                               # abrir el archivo de labels
fileContents = f.read()
fileContents = fileContents.split('\n')
for i in range(len(fileContents)-1):
  fileContents[i] = fileContents[i].split(',')
  naming_dict[fileContents[i][0]] = fileContents[i][1]
print(naming_dict)

{'id': 'breed', '000bec180eb18c7604dcecc8fe0dba07': 'boston_bull', '001513dfcb2ffafc82cccf4d8bbaba97': 'dingo', '001cdf01b096e06d78e9e5112d419397': 'pekinese', '00214f311d5d2247d5dfe4fe24b2303d': 'bluetick', '0021f9ceb3235effd7fcde7f7538ed62': 'golden_retriever', '002211c81b498ef88e1b40b9abf84e1d': 'bedlington_terrier', '00290d3e1fdd27226ba27a8ce248ce85': 'bedlington_terrier', '002a283a315af96eaea0e28e7163b21b': 'borzoi', '003df8b8a8b05244b1d920bb6cf451f9': 'basenji', '0042188c895a2f14ef64a918ed9c7b64': 'scottish_deerhound', '004396df1acd0f1247b740ca2b14616e': 'shetland_sheepdog', '0067dc3eab0b3c3ef0439477624d85d6': 'walker_hound', '00693b8bc2470375cc744a6391d397ec': 'maltese_dog', '006cc3ddb9dc1bd827479569fcdc52dc': 'bluetick', '0075dc49dab4024d12fafe67074d8a81': 'norfolk_terrier', '00792e341f3c6eb33663e415d0715370': 'african_hunting_dog', '007b5a16db9d9ff9d7ad39982703e429': 'wire-haired_fox_terrier', '007b8a07882822475a4ce6581e70b1f8': 'redbone', '007ff9a78eba2aebb558afea3a51c469': '

#### Crear el diccionario contador que comience en 0 para cada raza

In [0]:
breeds = naming_dict.values()
breed_set = set(breeds)         # set: Crea un nuevo conjunto iterable no ordenado a partir de breeds
counting_dict = {}
for i in breed_set:
  counting_dict[i] = 0

#### Guardar las imágenes con los nuevos nombres de archivo dados por el diccionario para train y test

Paciencia! Puede demorar unos minutos.

In [0]:
print('Guardando imágenes en labeled_train')
imgNumber = 0
for img in os.listdir('./train'):
  imgNumber = imgNumber + 1
  print('imágenes procesadas: ', imgNumber)
  imgName = img.split('.')[0]               # convierte '0913209.jpg' en '0913209'
  label = naming_dict[str(imgName)]         # obtiene la raza del diccionario
  counting_dict[label] += 1                 # aumenta la cuenta en el contador
  path = os.path.join('./train', img)
  saveName = './labeled_train/' + label + '-' + str(counting_dict[label]) + '.jpg' # nuevo nombre con raza y número
  image_data = np.array(Image.open(path))
  imageio.imwrite(saveName, image_data)     # guarda la imagen con el nuevo nombre

Guardando imágenes en labeled_train
imágenes procesadas:  1
imágenes procesadas:  2
imágenes procesadas:  3
imágenes procesadas:  4
imágenes procesadas:  5
imágenes procesadas:  6
imágenes procesadas:  7
imágenes procesadas:  8
imágenes procesadas:  9
imágenes procesadas:  10
imágenes procesadas:  11
imágenes procesadas:  12
imágenes procesadas:  13
imágenes procesadas:  14
imágenes procesadas:  15
imágenes procesadas:  16
imágenes procesadas:  17
imágenes procesadas:  18
imágenes procesadas:  19
imágenes procesadas:  20
imágenes procesadas:  21
imágenes procesadas:  22
imágenes procesadas:  23
imágenes procesadas:  24
imágenes procesadas:  25
imágenes procesadas:  26
imágenes procesadas:  27
imágenes procesadas:  28
imágenes procesadas:  29
imágenes procesadas:  30
imágenes procesadas:  31
imágenes procesadas:  32
imágenes procesadas:  33
imágenes procesadas:  34
imágenes procesadas:  35
imágenes procesadas:  36
imágenes procesadas:  37
imágenes procesadas:  38
imágenes procesadas:  3

In [16]:
print('Guardando imágenes en labeled_test')
imgNumber = 0
for img in os.listdir('./test'):
  print(img)
for img in os.listdir('./test'):
  imgNumber = imgNumber + 1
  print('imágenes procesadas: ', imgNumber)
  imgName = img.split('.')[0]               # convierte '0913209.jpg' en '0913209'
  label = naming_dict[str(imgName)]         # obtiene la raza del diccionario
  counting_dict[label] += 1                 # aumenta la cuenta en el contador
  path = os.path.join('./test', img)
  saveName = './labeled_test/' + label + '-' + str(counting_dict[label]) + '.jpg' # nuevo nombre con raza y número
  image_data = np.array(Image.open(path))
  imageio.imwrite(saveName, image_data)     # guarda la imagen con el nuevo nombre

Guardando imágenes en labeled_test
50f60661565a02c5b96f446f089832e6.jpg
99bd95316f9796c59072e84932ad33f9.jpg
c1d8d7185a8ddfc01b0625ddadfbd3b3.jpg
c174cfe16227d0d75aabe817a9bef945.jpg
c364569a4a1eba0dd82f4ae244e60aeb.jpg
31c45f6f4b1a1dd82a30541a25c2ad09.jpg
faaf9144e28a76de76ab05c59354f0fb.jpg
fc29d76d1e3cd3478e9c3121fa50749a.jpg
b081d636dd4407059f69622ce2d36362.jpg
84e3a6a1d53e886d97f65411489969f0.jpg
6e01c54252cfc40ecdfde9612d9e6414.jpg
93fa4194758960a2fffec91162579467.jpg
8651880bfa690882f2589c856fdea161.jpg
bdd302682f88523977c4f149d6f64224.jpg
e52a4321a8cb0dd3bcc8f4a68b75eee7.jpg
e68da487dabcd11874830a177e8b6218.jpg
27856322e5bd7791bb7c6d5bd31fd7c4.jpg
4384fc6887440d1d46dafbd0a6bdad71.jpg
ec036cf752a803815c0e8ecc6601b0e3.jpg
66479e171fbbd6f874e53c3a0864be1d.jpg
4338b655b34c0548b089958e1574d7f1.jpg
0d81ae5c4b549b678c83b83f5fbccbbb.jpg
2e66e5efdcb7043a05be411148971b79.jpg
9cf1938e43a7489a20a0960e359cb68d.jpg
85bb38d45f1efff6bc45453df671710a.jpg
3dd26fb7235e28f766e0dc705e7f5b07.jpg
607

KeyError: ignored

#### Modificar el tamaño de las imágenes

##### Estadísticas de los tamaños de las imágenes

Al ser las imágenes parte del dataset de imagenet, cada imagen tiene un tamaño distinto. Por lo que en la siguiente celda se calculan ciertas estadísticas con respecto a los tamaños: 
 - Altura promedio
 - Altura máxima
 - Alturna mínima
 - Ancho promedio
 - Ancho máximo
 - Ancho mínimo

 Esto servirá para establecer los tamaño finales de las imágenes de entrada.


In [0]:
DIR = './train'

def get_size_statistics():
    heights = []
    widths = []
    img_count = 0
    for img in os.listdir(DIR):
        path = os.path.join(DIR, img)
        if "DS_Store" not in path:
            data = np.array(Image.open(path))
            heights.append(data.shape[0])
            widths.append(data.shape[1])
            img_count += 1
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))

get_size_statistics()    

In [0]:
def label_img(name):
    word_label = name.split('-')[0]
    if word_label == 'golden_retriever': return [0, 0]
    elif word_label == 'shetland_sheepdog' : return [1, 0]
    return [0, 1]

##### Modificación del tamaño de las imágenes y data augmentation
Considerando los resultados de las estadísticas, finalmente se usará por diseño un tamaño de 300 x 300 para las imágenes de entrada. La celda siguiente realiza el trabajo de conversión de tamaño para todas las imágenes y además realiza un aumento de datos, al realizar un espejado de las imágenes de entrenamiento, para tener el doble de imágenes y obtener una mejor performace.

In [0]:
IMG_SIZE = 300
DIR = 'labeled_train'

def load_training_data():
    contador = 0
    img_count = 0
    train_data = []
    for img in os.listdir(DIR):
        label = label_img(img)
        path = os.path.join(DIR, img)
        print('label: ', label)
        if "DS_Store" not in path:
            img = Image.open(path)
            img = img.convert('L')
            img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
            train_data.append([np.array(img), label])

            # Basic Data Augmentation - Horizontal Flipping
            flip_img = Image.open(path)
            flip_img = flip_img.convert('L')
            flip_img = flip_img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
            flip_img = np.array(flip_img)
            flip_img = np.fliplr(flip_img)
            train_data.append([flip_img, label])
            
    shuffle(train_data)
    return train_data

##### Preparación de las imágenes de entrenamiento

La siguiente celda ejecuta la función definida en la anterior, cargando la variable *train_data* con las imágenes preprocesadas. Y muestra una en escala de grises.

In [0]:
train_data = load_training_data()
plt.imshow(train_data[43][0], cmap = 'gist_gray')

Se generan los arreglos *trainImages* y *trainLabels* a partir de *train_data*. Estos serán usados como entradas para el proceso de entrenamiento del modelo.

In [0]:
trainImages = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])

print(trainLabels[:10])

NameError: ignored

## Arquitectura de la red

### Importar los paquetes de Keras a utilizar

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
import numpy as np 

Using TensorFlow backend.


### Implementar la arquitectura del modelo

Se implementará un modelo con la siguiente arquitectura:

*Un bloque convolucional estará formado por una capa de convolución, una capa de pooling, y una capa de normalización*

1. Entrada: 300 x 300
2. Primer Bloque convolucional: 
*   Capa convolucional: 32 filtros de 3 x 3
*   Capa de pooling: MaxPooling de 2 x 2, stride de 1
*   Activación: ReLU
3. Segundo Bloque convolucional:
*   Capa convolucional: 64 filtros de 3 x 3
*   Capa de pooling: MaxPooling de 2 x 2, stride de 1
*   Activación: ReLU
4. Tercer Bloque convolucional: 
*   Capa convolucional: 64 filtros de 3 x 3
*   Capa de pooling: MaxPooling de 2 x 2, stride de 1
*   Activación: ReLU
5. Cuarto Bloque convolucional:
*   Capa convolucional: 96 filtros de 3 x 3
*   Capa de pooling: MaxPooling de 2 x 2, stride de 1
*   Activación: ReLU
6. Quinto Bloque convolucional:
*   Capa convolucional: 32 filtros de 3 x 3
*   Capa de pooling: MaxPooling de 2 x 2, stride de 1
*   Activación: ReLU
7. Primer Bloque totalmente conectado:
*   Capa totalmente conectada: 128 neuronas
*   Activación: ReLU
8. Capa de salida: 2 neuronas, regularización Softmax

In [0]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(2, activation = 'softmax'))













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compilación del modelo

Se realizará la compilación del modelo, utilizando como función de pérdida la definida mediante una entropía cruzada binaria, como optimizador la técnica adam, como métrica, la medida de precisión simple.



In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Entrenamiento del modelo

Se entrena el modelo con los arreglos *trainImages* y *trainLabels* definidos anteriormente, y con los siguientes parámetros:


*   Tamaño de batch: 50
*   Epochs: 5



In [0]:
model.fit(trainImages, trainLabels, batch_size = 50, epochs = 5, verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 13s 6ms/step - loss: nan - acc: 0.0040
Epoch 2/5
2000/2000 [==============================] - 11s 5ms/step - loss: nan - acc: 0.0040
Epoch 3/5
2000/2000 [==============================] - 11s 5ms/step - loss: nan - acc: 0.0040
Epoch 4/5
2000/2000 [==============================] - 11s 5ms/step - loss: nan - acc: 0.0040
Epoch 5/5
2000/2000 [==============================] - 11s 5ms/step - loss: nan - acc: 0.0040


### Cargar las imágenes de prueba

Con el modelo entrenado, se cargarán las imágenes de prueba en el arreglo *test_data* a partir del cual, con el mismo método que para las imágenes de entrenamiento, se obtendrán los arreglos de *testImages* y *testLabels* para evaluar el modelo.

In [15]:
# Test on Test Set
IMG_SIZE = 300
TEST_DIR = './test'
def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        label = label_img(img)
        path = os.path.join(TEST_DIR, img)
        print('img: ', img)
        print('label: ', label)
        if "DS_Store" not in path:
            img = Image.open(path)
            img = img.convert('L')
            img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
            test_data.append([np.array(img), label])
    shuffle(test_data)
    return test_data


test_data = load_test_data()    
plt.imshow(test_data[10][0], cmap = 'gist_gray')

img:  50f60661565a02c5b96f446f089832e6.jpg
label:  [0, 1]
img:  99bd95316f9796c59072e84932ad33f9.jpg
label:  [0, 1]
img:  c1d8d7185a8ddfc01b0625ddadfbd3b3.jpg
label:  [0, 1]
img:  c174cfe16227d0d75aabe817a9bef945.jpg
label:  [0, 1]
img:  c364569a4a1eba0dd82f4ae244e60aeb.jpg
label:  [0, 1]
img:  31c45f6f4b1a1dd82a30541a25c2ad09.jpg
label:  [0, 1]
img:  faaf9144e28a76de76ab05c59354f0fb.jpg
label:  [0, 1]
img:  fc29d76d1e3cd3478e9c3121fa50749a.jpg
label:  [0, 1]
img:  b081d636dd4407059f69622ce2d36362.jpg
label:  [0, 1]
img:  84e3a6a1d53e886d97f65411489969f0.jpg
label:  [0, 1]
img:  6e01c54252cfc40ecdfde9612d9e6414.jpg
label:  [0, 1]
img:  93fa4194758960a2fffec91162579467.jpg
label:  [0, 1]
img:  8651880bfa690882f2589c856fdea161.jpg
label:  [0, 1]
img:  bdd302682f88523977c4f149d6f64224.jpg
label:  [0, 1]
img:  e52a4321a8cb0dd3bcc8f4a68b75eee7.jpg
label:  [0, 1]
img:  e68da487dabcd11874830a177e8b6218.jpg
label:  [0, 1]
img:  27856322e5bd7791bb7c6d5bd31fd7c4.jpg
label:  [0, 1]
img:  4384fc68

KeyboardInterrupt: ignored

In [0]:
testImages = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data])

loss, acc = model.evaluate(testImages, testLabels, verbose = 0)
print(acc * 100)

## Tarea: Mejorar el modelo

Despues de haber obtenido una performance de aproximadamente el 75 %, existe lugar para mejorar el modelo. Se proponen 5 estrategias para realizarlo, aunque se puede usar cualquier otro método y/o una combinación de los propuestos.

1. Agregar más Dropout para evitar un overfitting en los datos de entrenamiento.
2. Modificar los parámetros de la red convolucional: agregar o quitar capas, modificar el número y tamaño de los filtros, o cambiar las funciones de activación. *Tener en cuenta los tamaños de los feature maps si se decide cambiar el número y tamaño de filtros*
3. Alterar la red totalmente conectada: agregando o quitando capas y/o cambiando el número de neuronas.
4. Aumento de datos: Durante el entrenamiento se utilizó una técnica básica de aumento de datos, un movimiento horizontal. Se pueden probar otros métodos tales como: otros movimientos, cambiar el tamaño, cambiar escalas de colores, entre otros.
5. Cortar las imágenes para obtener solo perros. El dataset de entrenamiento contiene diferentes imágenes donde aparecen perros. Sin embargo, en la misma imagen, aparecen otros objetos o elementos que pueden llevar a la red a confundir que detalles elementales conforman un perro. Una propuesta para mejorar sería corregir ciertas imagenes para solo clasificar imagenes de perros y no todo el resto.

In [0]:
## Todo: Realizar las mejores en esta celda, o modificar las celdas correspondientes y ordenarlas de manera que se deba correr el Notebook una sola vez. Se puede probar esto con yendo a Entorno de ejecución > Ejecutar todas